In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep

pd.options.display.max_columns = 5_000 # So I can see all the columns rather than just ...
pd.options.display.max_rows = 500
pd.options.display.max_colwidth = 1000

In [ ]:
driver = webdriver.Chrome(executable_path='../chromedriver')

In [263]:
allreviews = pd.DataFrame()
for pg in range(34):
    url = f'https://beerandbrewing.com/beer-reviews/?q=&hPP=30&idx=cbb_web_review_search&p={pg}'
    driver.get(url)
    sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    desc = soup.find_all('p',{'class':"hit-description"})
    reviews = []
    for brew in desc:
        beer = {}
        beer['description'] = brew.text
        reviews.append(beer)
    
    reviews = pd.DataFrame(reviews)
    
    beersoup = soup.find_all('div', {'class':['hit-content', 'col-xs-12', 'col-sm-12', 'col-md-10', 'text-wrapper']})
    
    names = []
    links = []
    for i in beersoup[6:]:
        if i.find('img',{'alt':True}) != None:
            img = i.find('img',{'alt':True})
            names.append(img['alt'])
        if i.find('a',{'href':True}) != None:
            link =  i.find('a',{'href':True})
            links.append(link['href'])
    
    beerlinks = []
    for i in range(0, len(links)):
        if i % 2 == 0 and (links[i].find('review') == True) and (links[i] not in beerlinks):
            beerlinks.append(links[i])
    
    reviews['names'] = names
    reviews['links'] = beerlinks
    allreviews = allreviews.append(reviews)

In [269]:
beerandbrewing = allreviews.reset_index().drop(columns='index')
beerandbrewing.to_csv('../../beerandbrewing_reviews.csv', index=False)

In [270]:
driver.close()